<a href="https://colab.research.google.com/github/BurakKizilkaya/Machine-Learning/blob/master/KIZILKAYA_malaria_cell_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNG 562 - HOMEWORK 2

# Detecting Malaria from Cell Images








## Convolution Neural Networks(CNN)

In [1]:
#importing train data

from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"burakkizilkaya","key":"7c7b6f9585e0232bc29c2fa80e888786"}'}

In [2]:
#if there is no kaggle directory, create a new one.
!mkdir -p ~/.kaggle
#copy kaggle.json file to kaggle directory
!cp kaggle.json ~/.kaggle/
#give owner read and write access permissions
!chmod 600 ~/.kaggle/kaggle.json
#check files in kaggle directory
!ls ~/.kaggle

kaggle.json


In [0]:
#install kaggle packages
!pip install -q kaggle
!pip install -q kaggle-cli

In [5]:
#download cell-images dataset
!kaggle datasets download -d iarunava/cell-images-for-detecting-malaria


 97% 328M/337M [00:05<00:00, 87.8MB/s]
100% 337M/337M [00:05<00:00, 66.8MB/s]


In [0]:
#unzip downloaded dataset file
!unzip cell-images-for-detecting-malaria.zip


In [7]:
# Using Keras Sequential Model

#import libraries
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator

cnn = Sequential()

Using TensorFlow backend.


In [8]:
#Adding 1st Convolution and Pooling Layer
cnn.add(Conv2D(32,kernel_size=(3,3),input_shape = (64, 64, 3), activation = 'relu'))
cnn.add(MaxPool2D(pool_size=(2,2)))

#Adding 2nd Convolution and Pooling Layer
cnn.add(Conv2D(32,kernel_size=(3,3),activation = 'relu'))
cnn.add(MaxPool2D(pool_size=(2,2)))

#Adding 3rd Convolution and Pooling Layer
cnn.add(Conv2D(32,kernel_size=(3,3), activation = 'relu'))
cnn.add(MaxPool2D(pool_size=(2,2)))

#Adding 4th Convolution and Pooling Layer
cnn.add(Conv2D(32,kernel_size=(3,3), activation = 'relu'))
cnn.add(MaxPool2D(pool_size=(2,2)))

#Flattening
cnn.add(Flatten())

cnn.add(Dense(units=128,activation='relu'))
cnn.add(Dense(units = 1, activation = 'sigmoid'))

#compile the model
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


In [9]:
# data generation from cell images
pdatagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
udatagen = ImageDataGenerator(rescale=1./255)
train = pdatagen.flow_from_directory('cell_images', target_size=(64,64), batch_size=32, class_mode = 'binary')
test = udatagen.flow_from_directory('cell_images', target_size=(64,64), batch_size=32, class_mode = 'binary')

Found 27558 images belonging to 2 classes.
Found 27558 images belonging to 2 classes.


In [10]:
# train the model on data 
acc = cnn.fit_generator(train, steps_per_epoch = 100, epochs = 30, validation_data = test, validation_steps = 50)

Instructions for updating:
Use tf.cast instead.
Epoch 1/30
100/100 [==============================] - 34s 335ms/step - loss: 0.6881 - acc: 0.5300 - val_loss: 0.6491 - val_acc: 0.6613
Epoch 2/30
100/100 [==============================] - 33s 328ms/step - loss: 0.5969 - acc: 0.6928 - val_loss: 0.5017 - val_acc: 0.7887
Epoch 3/30
100/100 [==============================] - 33s 326ms/step - loss: 0.3643 - acc: 0.8534 - val_loss: 0.2187 - val_acc: 0.9275
Epoch 4/30
100/100 [==============================] - 33s 328ms/step - loss: 0.2435 - acc: 0.9163 - val_loss: 0.2099 - val_acc: 0.9363
Epoch 5/30
100/100 [==============================] - 33s 326ms/step - loss: 0.2295 - acc: 0.9237 - val_loss: 0.1985 - val_acc: 0.9437
Epoch 6/30
100/100 [==============================] - 33s 329ms/step - loss: 0.2021 - acc: 0.9344 - val_loss: 0.1530 - val_acc: 0.9475
Epoch 7/30
100/100 [==============================] - 33s 329ms/step - loss: 0.1782 - acc: 0.9425 - val_loss: 0.1862 - val_acc: 0.9325
Epoch 8

# Transfer Learning

In [11]:
# RESNET50 from keras
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.python.keras import optimizers
import numpy as np

model = Sequential()

model.add(ResNet50(weights='imagenet'))
model.add(Dense(1, activation = 'relu'))

# dont train first layer (ResNet) model as it is already trained
model.layers[0].trainable = False

# data generation from cactus images
traindatagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
testdatagen = ImageDataGenerator(rescale=1./255)
train = traindatagen.flow_from_directory('cell_images', target_size=(224,224), batch_size=32, class_mode = 'binary')
test = testdatagen.flow_from_directory('cell_images', target_size=(224,224), batch_size=32, class_mode = 'binary')


model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

Found 27558 images belonging to 2 classes.
Found 27558 images belonging to 2 classes.


In [12]:
# epoch number can be increased to 30. It is kept as 1 since it takes too much time
acc = model.fit_generator(train, steps_per_epoch = 100, epochs = 1, validation_data = test, validation_steps = 50)

Epoch 1/1
100/100 [==============================] - 1443s 14s/step - loss: 1.7294 - acc: 0.5006 - val_loss: 1.4193 - val_acc: 0.4863
